### *New-York City Air Quality Analysis*


### *Business Problem*
*Problem Statement: Ferns and Petals Sales Analysis*
*You have been given a dataset from FNP (Ferns and Petals) that specializes in sending gifts for various occasions like Diwali, Raksha Bandhan, Holi, Valentine's Day, Birthdays, and Anniversaries. The dataset contains details about the products, orders, customers, and relevant dates.
Your task is to analyze this dataset to uncover key insights related to sales trends, customer behavior, and product performance.*

*Create a dashboard and answer the following key business questions to help the company improve its sales strategy and optimize customer satisfaction:*
1. *Total Revenue: Identify the overall revenue.*
2. *Average Order and Delivery Time: Evaluate the time taken for orders to be delivered.*
3. *Monthly Sales Performance: Examine how sales fluctuate across the months of 2023.*
4. *Top Products by Revenue: Determine which products are the top revenue generators.*
5. *Customer Spending Analysis: Understand how much customers are spending on average.*
6. *Sales Performance by Top 5 Product: Track the sales performance of top 5 products.*
7. *Top 10 Cities by Number of Orders: Find out which cities are placing the highest number of orders.*
8. *Order Quantity vs. Delivery Time: Analyze if higher order quantities impact delivery times.*
9. *Revenue Comparison Between Occasions: Compare revenue generated across different occasions.*
10. *Product Popularity by Occasion: Identify which products are most popular during specific occasions.*

*Author – Niranjan (Data Analyst)*

In [ ]:
import pandas as pd
import re

from datetime import datetime

pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv("Air_Quality.csv")

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
# List of columns to drop
cols_to_drop = [
    'Unique ID', 
    'Indicator ID',
    'Geo Type Name', 
    'Geo Join ID',
    'Message'
]

# Drop the columns
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df.columns

### *`Name`*

In [ ]:
df['Name'].unique()

In [ ]:
df['Name'].nunique()

In [ ]:
df['Name'].value_counts()

In [ ]:
df['Name'].isna().sum()

In [ ]:
df['Name'].dtype

In [ ]:
df['Name'] = df['Name'].str.strip()

In [ ]:
name_duplicates = df[df.duplicated('Name', keep=False)]  # Keep all duplicates
name_duplicates_sorted = name_duplicates.sort_values('Name')

# Get duplicate counts per name
duplicate_counts = df['Name'].value_counts().reset_index()
duplicate_counts.columns = ['Name', 'Count']
duplicate_counts = duplicate_counts[duplicate_counts['Count'] > 1]

print(f"Total duplicate name entries: {len(name_duplicates)}")
print(f"Unique duplicate names: {len(duplicate_counts)}")
print("\nTop duplicated names:")
print(duplicate_counts.head(10))

In [ ]:
# Convert 'Name' column to title case
df['Name'] = df['Name'].str.title()

# Verify changes
print("Sample standardized names:")
print(df['Name'].sample(5, random_state=1))

In [ ]:
df['Name'].value_counts()

In [ ]:
# Define the mapping rules (customize as needed)
name_mapping = {
    # Air Quality Indicators
    'Nitrogen Dioxide (No2)': 'NO2',
    'Fine Particles (Pm 2.5)': 'PM2.5',
    'Ozone (O3)': 'Ozone',
    'Asthma Emergency Departments Visits Due To Ozone': 'Asthma Visits',
    'Asthma Emergency Department Visits Due To Pm2.5': 'Asthma Visits',
    'Asthma Hospitalizations Due To Ozone': 'Asthma Hospitalizations',
    'Deaths Due To Pm2.5': 'Deaths',
    'Cardiovascular Hospitalizations Due To Pm2.5 (Age 40+)': 'Cardiovascular Hospitalizations',
    'Cardiac And Respiratory Deaths Due To Ozone': 'Deaths',
    'Respiratory Hospitalizations Due To Pm2.5 (Age 20+)': 'Respiratory Hospitalizations',
    'Annual Vehicle Miles Traveled': 'Vehicle Miles',
    'Annual Vehicle Miles Traveled (Trucks)': 'Vehicle Miles',
    'Annual Vehicle Miles Traveled (Cars)': 'Vehicle Miles',
    'Outdoor Air Toxics - Benzene': 'Outdoor Air Toxics',
    'Outdoor Air Toxics - Formaldehyde': 'Outdoor Air Toxics',
    'Boiler Emissions- Total Nox Emissions': 'Boiler Emissions',
    'Boiler Emissions- Total So2 Emissions': 'Boiler Emissions',
    'Boiler Emissions- Total Pm2.5 Emissions': 'Boiler Emissions',
}

# Apply mapping to create 'Name_Clean'
df['Name_Clean'] = df['Name'].map(name_mapping)

# For unmatched names, keep original (or set as 'Other')
df['Name_Clean'] = df['Name_Clean'].fillna('Other')  # or fillna(df['Name'])

# Verify the results
print("Top 10 Cleaned Names:")
print(df['Name_Clean'].value_counts().head(10))

In [ ]:
df['Name_Clean'].value_counts()

### *`Measure`*

In [ ]:
df['Measure'].unique()

In [ ]:
df['Measure'].nunique()

In [ ]:
df['Measure'].value_counts()

In [ ]:
# Trim whitespace from the 'Measure' column
df['Measure'] = df['Measure'].str.strip()

# Verify changes
print("Value counts after trimming:")
print(df['Measure'].value_counts())

In [ ]:
def standardize_measure(text):
    text = str(text).strip().title()
    text = text.replace('Per Km2', 'per km2')  # Handle special case
    text = text.replace('Km2', 'km2')          # Standardize units
    return text

# Standardize the actual data
df['Measure_Standardized'] = df['Measure'].apply(standardize_measure)

# Get counts from standardized data
counts = df['Measure_Standardized'].value_counts().reset_index()
counts.columns = ['Measure_Standardized', 'Actual_Count']

# Desired output measures (display names)
output_measures = [
    'Mean',
    'Number Per Km2',
    'Estimated Annual Rate (Age 18+)',
    'Million Miles',
    'Estimated Annual Rate',
    'Estimated Annual Rate (Under Age 18)',
    'Annual Average Concentration',
    'Estimated Annual Rate (Age 30+)'
]

# Create output DataFrame with standardized keys
df_output = pd.DataFrame({'Display_Measure': output_measures})
df_output['Measure_Standardized'] = df_output['Display_Measure'].apply(standardize_measure)

# Merge with actual counts
result = df_output.merge(counts, on='Measure_Standardized', how='left')
result['Count'] = result['Actual_Count'].fillna(0).astype(int)

# Final output with correct display names
final_output = result[['Display_Measure', 'Count']]
final_output.columns = ['Measure', 'Count']  # Rename columns

# Print formatted table
print(final_output.to_markdown(index=False, stralign='left'))

In [ ]:
mean_count = counts[counts['Measure_Standardized'] == 'Mean']['Actual_Count'].sum()
aac_count = counts[counts['Measure_Standardized'] == 'Annual Average Concentration']['Actual_Count'].sum()
combined_count = mean_count + aac_count

# Update counts for both measures in the result DataFrame
result.loc[result['Display_Measure'] == 'Mean', 'Count'] = combined_count
result.loc[result['Display_Measure'] == 'Annual Average Concentration', 'Count'] = combined_count

# Final output with correct display names
final_output = result[['Display_Measure', 'Count']]
final_output.columns = ['Measure', 'Count']  # Rename columns

# Print formatted table
print(final_output.to_markdown(index=False, stralign='left'))

In [ ]:
# Calculate combined count for Mean and Annual Average Concentration
mean_count = counts[counts['Measure_Standardized'] == 'Mean']['Actual_Count'].sum()
aac_count = counts[counts['Measure_Standardized'] == 'Annual Average Concentration']['Actual_Count'].sum()
combined_count = mean_count + aac_count

# Update count for Mean only (don't update Annual Average Concentration)
result.loc[result['Display_Measure'] == 'Mean', 'Count'] = combined_count

# Filter out the Annual Average Concentration row before final output
final_output = result[result['Display_Measure'] != 'Annual Average Concentration'][['Display_Measure', 'Count']]
final_output.columns = ['Measure', 'Count']  # Rename columns

# Print formatted table
print(final_output.to_markdown(index=False, stralign='left'))

### *`Measure Info`*

In [ ]:
df['Measure Info'].unique()

In [ ]:
df['Measure Info'].nunique()

In [ ]:
df['Measure Info'].value_counts()

In [ ]:
# Standardization function 
def standardize_measure(info):
    info = str(info).strip().lower()
    
    # Handle unit variations
    if any(unit in info for unit in ['µg/m3', 'âµg/m3', 'mcg/m3', 'ug/m3']):
        return 'mcg/m3'
    
    # Handle population rate variations
    if 'per 100,000' in info:
        if 'children' in info:
            return 'per 100,000 children'
        return 'per 100,000 adults'
    
    return info

# Apply standardization
df['Measure_Standardized'] = df['Measure Info'].apply(standardize_measure)

# Get value counts
measure_counts = df['Measure_Standardized'].value_counts().reset_index()
measure_counts.columns = ['Measure_Standardized', 'Count']

# Create desired output format
output_measures = [
    'ppb',
    'mcg/m3', 
    'per 100,000 adults',
    'per 100,000 children',
    'per square mile',
    'number'
]

# Merge and format
result = pd.DataFrame({'Measure Info': output_measures})
result = result.merge(
    measure_counts, 
    left_on='Measure Info', 
    right_on='Measure_Standardized', 
    how='left'
).drop(columns='Measure_Standardized')

# Fill NA with 0 and format numbers
result['Count'] = result['Count'].fillna(0).astype(int)
result['Count'] = result['Count'].apply(lambda x: f"{x:,}")

# Print formatted output
print(result.to_markdown(index=False, stralign='left'))

### *`Geo Place Name`*

In [ ]:
df['Geo Place Name'].unique()

In [ ]:
df['Geo Place Name'].nunique()

In [ ]:
df['Geo Place Name'].value_counts()

In [ ]:
pd.set_option('display.max_rows', None)
print(df['Geo Place Name'].value_counts())

In [ ]:
df['Geo Place Name'] = (
    df['Geo Place Name']
    .astype(str)          # Ensures all values are strings
    .str.strip()          # Trims spaces
    .str.title()          # Title Case
    .str.replace(r'\s+', ' ', regex=True)  # Fixes spacing
    .replace('^$', pd.NA, regex=True)  # Converts empty strings to NA
)

In [ ]:
df['Geo Place Name'] = df['Geo Place Name'].str.title()

In [ ]:
df['Geo Place Name'].value_counts()

### *`Time Period`*

In [ ]:
df['Time Period'].unique()

In [ ]:
df['Time Period'].nunique()

In [ ]:
df['Time Period'].value_counts()

In [ ]:
df['Time Period'] = df['Time Period'].str.strip().str.title()

In [ ]:
df['Time Period'].value_counts()

In [ ]:
df['Season'] = df['Time Period'].apply(
    lambda x: 'Annual' if x.startswith('Annual') else 
              x.split()[0] if any(season in x for season in ['Winter', 'Summer']) else 'Annual'
)

In [ ]:
df['Season'].value_counts()

In [ ]:
df.shape

In [ ]:
def extract_period_year(period):
    if period.startswith('Annual Average'):
        return period.split()[-1]
    elif any(season in period for season in ['Winter', 'Summer']):
        return ' '.join(period.split()[1:])
    else:
        return period

df['Period_Year'] = df['Time Period'].apply(extract_period_year)

In [ ]:
df['Period_Year'].value_counts()

In [ ]:
def clean_period_year(period):
    if '-' not in period:
        return period
    start, end = period.split('-')
    if len(end) == 2:
        end = start[:2] + end
    return f"{start}-{end}"

df['Period_Year'] = df['Period_Year'].apply(clean_period_year)

In [ ]:
df['Period_Year'].value_counts()

In [ ]:
df['Period_Year'].isnull().sum()

### *`Start_Date`*

In [ ]:
df["Start_Date"].unique()

In [ ]:
df["Start_Date"].nunique()

In [ ]:
df["Start_Date"].value_counts()

In [ ]:
def convert_date_format(date_str):
    try:
        # First try parsing as DD/MM/YYYY
        dt = datetime.strptime(date_str, '%d/%m/%Y')
        return dt.strftime('%d/%m/%Y')  # Return in desired format
        
    except ValueError:
        try:
            # If that fails, try parsing as MM/DD/YYYY
            dt = datetime.strptime(date_str, '%m/%d/%Y')
            # Convert to DD/MM/YYYY format
            return dt.strftime('%d/%m/%Y')
            
        except ValueError:
            # If both fail, return the original string (or handle as needed)
            return date_str

# Apply the conversion to your column
df['Start_Date'] = df['Start_Date'].astype(str).apply(convert_date_format)

# Verify the conversion
print("Value counts after conversion:")
print(df['Start_Date'].value_counts().head(20))

In [ ]:
df["Start_Date"].value_counts()

In [ ]:
df["Start_Date"].isna().sum()

### *`Data Value`*

In [ ]:
df['Data Value'].unique()

In [ ]:
df['Data Value'].nunique()

In [ ]:
df['Data Value'].describe()

In [ ]:
df['Data Value'] = df['Data Value'].round(2)

In [ ]:
df['Data Value'].describe()

In [ ]:
df[df['Data Value'] < 0]

In [ ]:
df[df['Data Value'] > 100] 

In [ ]:
df['Data Value'].dtype

In [ ]:
df.columns